In [1]:
import numpy as np
import pandas as pd

import calendar
import datetime as dt
from collections import Counter

In [2]:
zeile = pd.read_csv(r'Einkaufszeile.csv', on_bad_lines='skip', sep=';',low_memory=False)
zeile.head()

,Attached to Line No_,Auto Charge Doc_ Line No_,Auto Charge Doc_ No_,Billing Entry No_,Billing Reference No_,Blanket Order Line No_,Blanket Order No_,Budgeted FA No_,Buy-from Vendor No_,Contract Line No_,...,VAT Base Amount,VAT Bus_ Posting Group,VAT Calculation Type,VAT Difference,VAT Identifier,VAT Prod_ Posting Group,VAT _,VDP Code,Variant Code,timestamp
0,0,0,NaN,0,NaN,0,NaN,NaN,K04148,0,...,"3950,00000000000000000000",IL,0,"0,00000000000000000000",14.0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052ADF4C4
1,0,0,NaN,0,NaN,0,NaN,NaN,K04138,0,...,"5650,61000000000000000000",EU,1,"0,00000000000000000000",14.0,MWST19,"0,00000000000000000000",NaN,NaN,00000000572F7357
2,0,0,NaN,0,NaN,0,NaN,NaN,K03105,0,...,"62,40000000000000000000",IL,0,"0,00000000000000000000",14.0,MWST19,"19,00000000000000000000",NaN,NaN,00000000572F78DF
3,0,0,NaN,0,NaN,0,NaN,NaN,K02739,0,...,"7496,60000000000000000000",IL,0,"0,00000000000000000000",14.0,MWST19,"19,00000000000000000000",NaN,NaN,0000000055C7E662
4,0,0,NaN,0,NaN,0,NaN,NaN,K02739,0,...,"3567,73000000000000000000",IL,0,"0,00000000000000000000",14.0,MWST19,"19,00000000000000000000",NaN,NaN,0000000055C7E663


In [3]:
zeile.shape

(4416, 249)

In [4]:
# drop columns where all NaN values
zeile=zeile.dropna(axis=1, how='all')
zeile.shape

(4416, 197)

In [5]:
# drop columns where all 0 values
zeile=zeile.loc[:, (zeile != 0).any(axis=0)]
zeile.shape

(4416, 153)

In [6]:
# drop columns where over 50% of values are null values
zeile=zeile.loc[:, zeile.isnull().mean()<0.5]
zeile.shape

(4416, 141)

In [7]:
# get list with column names
list(zeile.columns)

['Blanket Order Line No_',
 'Buy-from Vendor No_',
 'Cross-Reference Type No_',
 'Demand Query Line No_',
 'Document No_',
 'Line No_',
 'No_',
 'Pay-to Vendor No_',
 'Pos_ No_',
 'Prod_ Order Line No_',
 'Routing Reference No_',
 'A_ Rcd_ Not Inv_ Ex_ VAT (LCY)',
 'Allow Invoice Disc_',
 'Allow Item Charge Assignment',
 'Amount Including VAT',
 'Amount',
 'Amt_ Rcd_ Not Invoiced (LCY)',
 'Amt_ Rcd_ Not Invoiced',
 'Bin Code',
 'Blanket Order valid from',
 'Blanket Order valid to',
 'Completely Received',
 'Contract Amount',
 'Cross-Reference Type',
 'Description 2',
 'Description',
 'Dimension Set ID',
 'Direct Unit Cost Price Factor',
 'Direct Unit Cost',
 'Document Type',
 'Expected Receipt Date',
 'Expected Receipt Time',
 'FA Posting Date',
 'Fixed Quantity',
 'Gen_ Bus_ Posting Group',
 'Gen_ Prod_ Posting Group',
 'Gross Weight',
 'Indirect Cost _',
 'Inv_ Disc_ Amount to Invoice',
 'Inv_ Discount Amount',
 'Invoice Quantity (Item Charge)',
 'Item Category Code',
 'Job Currency 

In [8]:
# drop columns that don't seem relevant to our challenge
to_drop=['Blanket Order Line No_',  'Buy-from Vendor No_', 'Cross-Reference Type No_', 'Demand Query Line No_', 'Document No_', 'Line No_', 'Pay-to Vendor No_',
 'Pos_ No_', 'Prod_ Order Line No_', 'Routing Reference No_', 'A_ Rcd_ Not Inv_ Ex_ VAT (LCY)', 'Allow Invoice Disc_', 'Allow Item Charge Assignment',
 'Bin Code', 'Blanket Order valid from', 'Blanket Order valid to', 'Contract Amount', 'Cross-Reference Type', 'Description 2', 'Dimension Set ID', 'Direct Unit Cost Price Factor',
 'Direct Unit Cost', 'Document Type', 'Fixed Quantity', 'Gen_ Bus_ Posting Group', 'Gen_ Prod_ Posting Group', 'Gross Weight', 'Indirect Cost _', 'Inv_ Disc_ Amount to Invoice',
 'Inv_ Discount Amount', 'Job Currency Factor', 'Job Line Amount (LCY)', 'Job Line Amount', 'Job Line Disc_ Amount (LCY)', 'Job Line Discount Amount', 'Job Line Discount _',
 'Job Remaining Qty_ (Base)', 'Job Remaining Qty_', 'Job Total Price (LCY)', 'Job Unit Price (LCY)', 'Job Unit Price', 'Line Amount', 'Line Discount Amount',
 'Line Discount _', 'Max Order Quantity', 'Min Order Quantity', 'Net Weight', 'Outstanding Amount (LCY)', 'Outstanding Amount', 'Outstanding Amt_ Ex_ VAT (LCY)',
 'Outstanding Qty_ (Base)', 'Outstanding Quantity', 'Overhead Rate', 'Pack Sample Quantity (Base)', 'Pack Sample Quantity', 'Planned Receipt Date',
 'Planning Flexibility', 'Posting Group', 'Prepayment Amount', 'Prepayment VAT Difference', 'Prepayment VAT _', 'Prepayment _', 'Prepmt Amt Deducted',
 'Prepmt Amt to Deduct', 'Prepmt VAT Diff_ Deducted', 'Prepmt VAT Diff_ to Deduct', 'Prepmt_ Amount Inv_ (LCY)', 'Prepmt_ Amount Inv_ Incl_ VAT',
 'Prepmt_ Amt_ Incl_ VAT', 'Prepmt_ Amt_ Inv_', 'Prepmt_ Line Amount', 'Prepmt_ VAT Amount Inv_ (LCY)', 'Prepmt_ VAT Base Amt_', 'Price Factor',
 'Profit _', 'Promised Receipt Date', 'Recalculate Invoice Disc_', 'Ret_ Qty_ Shpd Not Invd_(Base)', 'Return Qty_ Shipped (Base)',
 'Return Qty_ Shipped Not Invd_', 'Return Qty_ Shipped', 'Return Qty_ to Ship (Base)', 'Return Qty_ to Ship', 'Return Shpd_ Not Invd_ (LCY)',
 'Return Shpd_ Not Invd_', 'Returns Deferral Start Date', 'Safety Lead Time', 'Salvage Value', 'Shortcut Dimension 1 Code', 'Shortcut Dimension 2 Code',
 'Unit Cost (LCY)', 'Unit Price (LCY)', 'Unit Volume', 'Unit of Measure (Cross Ref_)', 'Unit of Measure Code', 'Unit of Measure', 'Units per Parcel',
 'VAT Base Amount', 'VAT Bus_ Posting Group', 'VAT Calculation Type', 'VAT Difference', 'VAT Identifier', 'VAT Prod_ Posting Group', 'VAT _', 'timestamp',
'Amt_ Rcd_ Not Invoiced (LCY)', 'Amt_ Rcd_ Not Invoiced','FA Posting Date','Qty_ to Receive (Base)','Quantity (Base)','Unit Cost','Description',
 'Lead Time Calculation', 'Expected Receipt Time','Original Commission Quantity','Qty_ Invoiced (Base)', 'Qty_ Rcd_ Not Invoiced (Base)','Qty_ Rcd_ Not Invoiced',
 'Qty_ Received (Base)', 'Qty_ per Unit of Measure', 'Qty_ to Invoice (Base)', 'Qty_ to Invoice', 'Qty_ to Receive','Item Category Code','Invoice Quantity (Item Charge)',
 'Job Total Price','Order Quantity','Original Date','Completely Received','Type']
zeile.drop(to_drop, inplace=True, axis=1)

In [9]:
list(zeile.columns)

['No_',
 'Amount Including VAT',
 'Amount',
 'Expected Receipt Date',
 'Location Code',
 'Order Date',
 'Quantity Invoiced',
 'Quantity Received',
 'Quantity',
 'Really Receipt DateTime',
 'Requested Receipt Date']

In [10]:
zeile.head()

,No_,Amount Including VAT,Amount,Expected Receipt Date,Location Code,Order Date,Quantity Invoiced,Quantity Received,Quantity,Really Receipt DateTime,Requested Receipt Date
0,HD00034020,"4700,50000000000000000000","3950,00000000000000000000",11.01.2019 00:00:00,B-01,25.04.2018 00:00:00,"250,00000000000000000000","250,00000000000000000000","250,00000000000000000000",01.01.1753 00:00:00,11.01.2019 00:00:00
1,WA00497656,"5650,61000000000000000000","5650,61000000000000000000",28.02.2019 00:00:00,B-01,21.08.2018 00:00:00,"1218,00000000000000000000","1218,00000000000000000000","1218,00000000000000000000",01.01.1753 00:00:00,28.02.2019 00:00:00
2,MV00300385,"74,26000000000000000000","62,40000000000000000000",27.05.2019 00:00:00,B-01,19.10.2018 00:00:00,"1000,00000000000000000000","1000,00000000000000000000","1000,00000000000000000000",01.01.1753 00:00:00,29.11.2018 00:00:00
3,17000162,"8920,95000000000000000000","7496,60000000000000000000",29.11.2019 00:00:00,B-01,05.06.2019 00:00:00,"2000,00000000000000000000","2000,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,29.11.2019 00:00:00
4,17000162,"4245,60000000000000000000","3567,73000000000000000000",10.01.2020 00:00:00,B-01,05.06.2019 00:00:00,"904,00000000000000000000","904,00000000000000000000","904,00000000000000000000",01.01.1753 00:00:00,10.01.2020 00:00:00


In [11]:
# make feature names consistent
zeile.columns = zeile.columns.str.replace(" ", "_").str.lower()
zeile=zeile.rename(columns={"no_": "purchase_no"})
list(zeile.columns)

['purchase_no',
 'amount_including_vat',
 'amount',
 'expected_receipt_date',
 'location_code',
 'order_date',
 'quantity_invoiced',
 'quantity_received',
 'quantity',
 'really_receipt_datetime',
 'requested_receipt_date']

In [12]:
# drop unnecessary characters to enable typecasting
zeile.expected_receipt_date = zeile.expected_receipt_date.map(lambda x: str(x)[:-9])
zeile.order_date = zeile.order_date.map(lambda x: str(x)[:-9])
zeile.really_receipt_datetime = zeile.really_receipt_datetime.map(lambda x: str(x)[:-9])
zeile.requested_receipt_date = zeile.requested_receipt_date.map(lambda x: str(x)[:-9])
zeile.amount_including_vat = zeile.amount_including_vat.map(lambda x: str(x)[:-18])
zeile.amount = zeile.amount.map(lambda x: str(x)[:-18])
zeile.quantity_invoiced = zeile.quantity_invoiced.map(lambda x: str(x)[:-21])
zeile.quantity_received = zeile.quantity_received.map(lambda x: str(x)[:-21])
zeile.quantity = zeile.quantity.map(lambda x: str(x)[:-21])

zeile.head(3)

,purchase_no,amount_including_vat,amount,expected_receipt_date,location_code,order_date,quantity_invoiced,quantity_received,quantity,really_receipt_datetime,requested_receipt_date
0,HD00034020,"4700,50","3950,00",11.01.2019,B-01,25.04.2018,250,250,250,01.01.1753,11.01.2019
1,WA00497656,"5650,61","5650,61",28.02.2019,B-01,21.08.2018,1218,1218,1218,01.01.1753,28.02.2019
2,MV00300385,"74,26","62,40",27.05.2019,B-01,19.10.2018,1000,1000,1000,01.01.1753,29.11.2018


In [13]:
zeile.dtypes

purchase_no                object
amount_including_vat       object
amount                     object
expected_receipt_date      object
location_code              object
order_date                 object
quantity_invoiced          object
quantity_received          object
quantity                   object
really_receipt_datetime    object
requested_receipt_date     object
dtype: object

In [14]:
zeile.shape

(4416, 11)

In [15]:
# Dates in the really_receipt_datetime columns with a value of 1753-01-01 mean the field is blank or there's been an error in the input.
# we will filter out these rows (1,236 from 6012)
zeile = zeile[zeile['really_receipt_datetime']!= '01.01.1753']
zeile.shape

(3621, 11)

In [16]:
list(zeile.columns)

['purchase_no',
 'amount_including_vat',
 'amount',
 'expected_receipt_date',
 'location_code',
 'order_date',
 'quantity_invoiced',
 'quantity_received',
 'quantity',
 'really_receipt_datetime',
 'requested_receipt_date']

In [17]:
zeile.head(3)

,purchase_no,amount_including_vat,amount,expected_receipt_date,location_code,order_date,quantity_invoiced,quantity_received,quantity,really_receipt_datetime,requested_receipt_date
18,MS00203015,"10973,75","9460,13",10.09.2020,B-01,02.07.2020,1250,1250,1250,22.07.2021,10.09.2020
19,MS00203015,"11257,55","9460,13",12.03.2021,B-01,24.09.2019,1250,1250,1250,22.07.2021,15.03.2021
20,DA00114007,"9814,21","8247,23",06.08.2021,B-01,06.08.2021,2415,2415,2415,22.07.2021,06.08.2021


In [18]:
# replace the commas for dots so we can typecast to float
zeile.amount_including_vat = zeile.amount_including_vat.str.replace(",", ".").str.lower()
zeile.amount = zeile.amount.str.replace(",", ".").str.lower()
zeile.head(30)

,purchase_no,amount_including_vat,amount,expected_receipt_date,location_code,order_date,quantity_invoiced,quantity_received,quantity,really_receipt_datetime,requested_receipt_date
18,MS00203015,10973.75,9460.13,10.09.2020,B-01,02.07.2020,1250,1250,1250,22.07.2021,10.09.2020
19,MS00203015,11257.55,9460.13,12.03.2021,B-01,24.09.2019,1250,1250,1250,22.07.2021,15.03.2021
20,DA00114007,9814.21,8247.23,06.08.2021,B-01,06.08.2021,2415,2415,2415,22.07.2021,06.08.2021
28,RA00000360,99.36,83.50,29.11.2019,A-01,08.10.2019,5000,5000,5000,14.01.2020,28.10.2019
29,RA00000370,61.59,51.75,29.11.2019,A-01,08.10.2019,1500,1500,1500,14.01.2020,28.10.2019
30,RA00000380,1467.91,1233.54,16.01.2020,A-01,26.09.2019,9900,9900,9900,14.01.2020,28.10.2019
31,RA00000330,9007.11,7569.00,12.12.2019,A-01,08.10.2019,18000,18000,18000,14.01.2020,12.12.2019
37,HD00105007,4099.79,3445.20,30.07.2021,B-01,12.02.2021,1080,1080,1080,29.07.2021,02.07.2021
40,RA00000245,1313.62,1103.88,18.02.2020,A-01,08.01.2020,2322,2322,2322,23.01.2020,22.01.2020
41,RA00000360,596.19,501.00,24.01.2020,A-01,02.01.2020,30000,30000,30000,23.01.2020,22.01.2020


In [19]:
# typecast
zeile = zeile.astype({'expected_receipt_date': 'datetime64', 'order_date': 'datetime64','really_receipt_datetime': 'datetime64',
'requested_receipt_date': 'datetime64','amount_including_vat': 'float64','amount': 'float64','quantity_invoiced': 'int64',
'quantity_received': 'int64','quantity': 'int64',})
zeile.dtypes

purchase_no                        object
amount_including_vat              float64
amount                            float64
expected_receipt_date      datetime64[ns]
location_code                      object
order_date                 datetime64[ns]
quantity_invoiced                   int64
quantity_received                   int64
quantity                            int64
really_receipt_datetime    datetime64[ns]
requested_receipt_date     datetime64[ns]
dtype: object

In [20]:
zeile.shape

(3621, 11)

In [21]:
# drop duplicates
zeile.drop_duplicates(keep='first')
zeile.shape

(3621, 11)

In [22]:
# check for null values
zeile.isna().sum()

purchase_no                0
amount_including_vat       0
amount                     0
expected_receipt_date      0
location_code              0
order_date                 0
quantity_invoiced          0
quantity_received          0
quantity                   0
really_receipt_datetime    0
requested_receipt_date     0
dtype: int64

In [23]:
zeile.head()

,purchase_no,amount_including_vat,amount,expected_receipt_date,location_code,order_date,quantity_invoiced,quantity_received,quantity,really_receipt_datetime,requested_receipt_date
18,MS00203015,10973.75,9460.13,2020-10-09,B-01,2020-02-07,1250,1250,1250,2021-07-22,2020-10-09
19,MS00203015,11257.55,9460.13,2021-12-03,B-01,2019-09-24,1250,1250,1250,2021-07-22,2021-03-15
20,DA00114007,9814.21,8247.23,2021-06-08,B-01,2021-06-08,2415,2415,2415,2021-07-22,2021-06-08
28,RA00000360,99.36,83.50,2019-11-29,A-01,2019-08-10,5000,5000,5000,2020-01-14,2019-10-28
29,RA00000370,61.59,51.75,2019-11-29,A-01,2019-08-10,1500,1500,1500,2020-01-14,2019-10-28


In [24]:
# export as csv file
zeile.to_csv(r'zeile_clean.csv', index=False);